In [1]:
import pandas as pd
import requests
from subprocess import Popen, PIPE
import os

In [2]:
df = pd.read_csv('annotations.tsv', delimiter='\t')

In [3]:
val_start = df.columns.tolist().index('Value Totals')

In [4]:
df.columns = [c.strip() for c in df.columns[:val_start]] + df.iloc[1, val_start:].values.tolist()

In [5]:
df.dropna(subset=['Title'], inplace=True)

In [6]:
filter_rows = df[['Title']].merge(df.iloc[:, list(range(val_start, df.shape[1]))].dropna().astype(int), right_index=True, left_index=True)

In [12]:
filter_rows.groupby('Title').mean()

,Novelty,Simplicity,Generalization,Flexibility/Extensibility,Robustness,Realistic output,Formal description/analysis,Theoretical guarantees,Approximation,Quantitative evidence (e.g. experiments),...,Non-maleficence,Justice,Respect for Persons,Autonomy (power to decide),Explicability,Respect for Law and public interest,Security,Easy to work with,Realistic world model,Fast
Title,,,,,,,,,,,,,,,,,,,,,
3D Object Recognition with Deep Belief Nets,1.5,0.0,1.5,0.5,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A Convergence Theory for Deep Learning via Over-Parameterization,0.0,4.0,10.0,6.0,0.0,0.0,8.0,11.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A dual coordinate descent method for large-scale linear SVM,1.0,4.0,0.0,1.0,0.0,0.0,2.0,2.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A unified architecture for natural language processing:deep neural networks with multitask learning,2.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
Addressing Function Approximation Error in Actor-Critic Methods,5.0,0.0,3.0,1.0,0.0,0.0,0.0,0.0,1.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ViLBERT: Pretraining Task-Agnostic Visiolinguistic Representations for Vision-and-Language Tasks,2.0,0.0,5.0,4.0,0.0,0.0,0.0,0.0,0.0,6.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
Video-to-Video Synthesis,3.5,0.0,2.5,2.5,0.5,10.0,0.5,0.0,0.5,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
Which Training Methods for GANs do actually Converge?,2.0,6.0,0.5,0.0,0.5,1.0,4.0,8.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [20]:
labeled = filter_rows.groupby('Title').apply(lambda s: (s > 0).all()).astype(int).reset_index()

In [21]:
data = labeled.merge(df[['Title', 'URL']].drop_duplicates(), on='Title')

In [22]:
def pdf_link(d):
    if 'icml2009' in d:
        last = d.split('/')[-1]
        return 'https://icml.cc/Conferences/2009/papers/' + last
    if 'pdf' not in d:
        if 'arxiv' in d:
            parts = d.split('/')
            parts[-2] = 'pdf'
            return '/'.join(parts)
        elif 'html' in d:
            return 'http://proceedings.mlr.press/v80/kim18b/kim18b.pdf'
        else:
            return d + '.pdf'
    return d

In [23]:
data.URL = data.URL.apply(pdf_link)

In [24]:
def download_pdf(row):
    url = row.URL
    filename = 'papers/' + row.Title + '.pdf'
    try:
        response = requests.get(url)
        response.raise_for_status()
        with open(filename, 'wb') as f:
            f.write(response.content)
    except requests.exceptions.HTTPError as err:
        print(f"HTTP Error occurred: {err}")
    except Exception as e:
        print(f"An error occurred: {e}")

In [25]:
data.apply(download_pdf, axis=1)

0     None
1     None
2     None
3     None
4     None
      ... 
92    None
93    None
94    None
95    None
96    None
Length: 97, dtype: object

In [26]:
data

,Title,Novelty,Simplicity,Generalization,Flexibility/Extensibility,Robustness,Realistic output,Formal description/analysis,Theoretical guarantees,Approximation,...,Justice,Respect for Persons,Autonomy (power to decide),Explicability,Respect for Law and public interest,Security,Easy to work with,Realistic world model,Fast,URL
0,3D Object Recognition with Deep Belief Nets,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,http://papers.nips.cc/paper/3872-3d-object-rec...
1,A Convergence Theory for Deep Learning via Ove...,0,1,1,1,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,https://arxiv.org/pdf/1811.03962v3.pdf
2,A dual coordinate descent method for large-sca...,1,1,0,1,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,http://icml2008.cs.helsinki.fi/papers/166.pdf
3,A unified architecture for natural language pr...,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,http://icml2008.cs.helsinki.fi/papers/391.pdf
4,Addressing Function Approximation Error in Act...,1,0,1,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,https://arxiv.org/pdf/1802.09477
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,ViLBERT: Pretraining Task-Agnostic Visiolingui...,1,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,http://papers.nips.cc/paper/8297-vilbert-pretr...
93,Video-to-Video Synthesis,1,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,http://papers.nips.cc/paper/7391-video-to-vide...
94,Which Training Methods for GANs do actually Co...,1,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,http://www.nowozin.net/sebastian/papers/mesche...
95,Wide Neural Networks of Any Depth Evolve as Li...,1,1,1,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,http://papers.nips.cc/paper/9063-wide-neural-n...


In [27]:
def pdf_to_text(pdf_path):
    process = Popen(['pdftotext', pdf_path, '-'], stdout=PIPE, stderr=PIPE)
    stdout, stderr = process.communicate()
    if process.returncode == 0:
        return stdout.decode('utf-8')
    else:
        print(f"Error processing {pdf_path}: {stderr}")
        return None

In [28]:
papers_dir = 'papers'
data['Text'] = ''
for index, row in data.iterrows():
    pdf_file_name = row['Title'] + '.pdf'
    pdf_path = os.path.join(papers_dir, pdf_file_name)
    if os.path.exists(pdf_path):
        data.at[index, 'Text'] = pdf_to_text(pdf_path)
    else:
        print(f"File not found: {pdf_path}")

In [29]:
data.to_csv('values_ml_papers.csv', index=False)